# Create a synthetic version of your own CSV or DataFrame

This blueprint utilizes Gretel's premium SDKs to create a synthetic version of your own data. Our SDKs create automatic data validators to help ensure the data generated has the same semantics as the source data. Additionally, the SDKs do autmoatic header clustering to help maintain statistical relations between columns.

In [ ]:
%%capture

!pip install -U gretel-client gretel-synthetics pandas

In [ ]:
# Load your Gretel API key. You can acquire this from the Gretel Console 
# @ https://console.gretel.cloud

import pandas as pd
from gretel_client import get_cloud_client

pd.set_option('max_colwidth', None)

client = get_cloud_client(prefix="api", api_key="prompt")
client.install_packages()

In [ ]:
# Load and preview dataset

import pandas as pd

dataset_path = 'https://gretel-public-website.s3-us-west-2.amazonaws.com/datasets/healthcare-analytics-vidhya/train_data.csv'
nrows = 10000  # We will use this later when generating data
training_df = pd.read_csv(dataset_path, nrows=nrows)
training_df

In [ ]:
# Create the Gretel Synthtetics Training / Model Configuration

from pathlib import Path

checkpoint_dir = str(Path.cwd() / "checkpoints")

config_template = {
    "checkpoint_dir": checkpoint_dir,
    "vocab_size": 20000,
    "overwrite": True
}

In [ ]:
# Capture transient import errors in Google Colab

try:
    from gretel_helpers.synthetics import SyntheticDataBundle
except FileNotFoundError:
    from gretel_helpers.synthetics import SyntheticDataBundle

In [ ]:
# Create a Gretel Synthetic Data Bundle

from gretel_helpers.synthetics import create_df, SyntheticDataBundle

model = SyntheticDataBundle(
    training_df=training_df,
    delimiter=None, # if ``None``, it will try and automatically be detected, otherwise you can set it
    auto_validate=True, # build record validators that learn per-column, these are used to ensure generated records have the same composition as the original
    synthetic_config=config_template, # the config for Synthetics
)

In [ ]:
model.build()

In [ ]:
model.train()

In [ ]:
# num_lines: how many rows to generate
# max_invalid: the number of rows that do not pass semantic validation, if this number is exceeded, training will
# stop
model.generate(num_lines=nrows, max_invalid=nrows)

In [ ]:
model.get_synthetic_df()

In [ ]:
# Generate report that shows the statistical performance between the training and synthetic data
import IPython

report_path = './report.html'
model.generate_report(report_path=report_path)
IPython.display.HTML(filename=report_path)

In [ ]:
# Optionally save your model

model.save("my_model.tar.gz")

In [ ]:
# Save synthetic dataframe locally and to a private Gretel project 

df = model.get_synthetic_df()
df.to_csv('synthetic-data.csv', index=False)

# Publish newly created synthetic data to a new private Gretel project 
project = client.get_project(display_name="Blueprint: Create Synthetic Data", create=True)
project.send_dataframe(df, detection_mode="all")
print(f"View this project at: {project.get_console_url()}")